In [15]:
import imaplib

def read(sender_of_interest=None):
    imap = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    imap.login('remboplas@gmail.com', 'z928wwM4')
    imap.select('INBOX')
    if sender_of_interest:
        status, response = imap.uid('search', None, 'UNSEEN', 'FROM {0}'.format(sender_of_interest))
    else:
        status, response = imap.uid('search', None, 'UNSEEN')
    if status == 'OK':
        unread_msg_nums = response[0].split()
    else:
        unread_msg_nums = []
    data_list = []
    for e_id in unread_msg_nums:
        e_id = e_id.decode('utf-8')
        _, response = imap.uid('fetch', e_id, '(RFC822)')
        html = response[0][1].decode('utf-8')
        email_message = email.message_from_string(html)
        data_list.append(email_message.get_payload())
    for elem in data_list:
        clean_html(elem)

: 

In [16]:
read()

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'

In [3]:
import os
import base64
import google.auth
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import email

# Если вы хотите иметь доступ только к почте, используйте этот scope
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    creds = None
    # Файл token.json сохраняет ваши учётные данные для повторного использования
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Если учётных данных нет или они устарели, проведём авторизацию
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Сохраняем токены для последующего использования
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

def get_unread_messages(service):
    # Используем Gmail API для получения списка непрочитанных сообщений
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], q='is:unread').execute()
    messages = results.get('messages', [])
    
    email_data = []
    
    if not messages:
        print('No unread messages found.')
    else:
        print(f'Found {len(messages)} unread messages.')
        for msg in messages:
            msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
            msg_payload = msg_data['payload']
            headers = msg_payload.get("headers")
            
            for header in headers:
                if header['name'] == 'From':
                    from_header = header['value']
                    print(f'From: {from_header}')
            
            # Получаем тело сообщения
            if 'parts' in msg_payload:
                for part in msg_payload['parts']:
                    if part['mimeType'] == 'text/plain':
                        body = part['body']['data']
                        decoded_body = base64.urlsafe_b64decode(body).decode('utf-8')
                        print(f'Message: {decoded_body}')
                        email_data.append({
                            'from': from_header,
                            'body': decoded_body
                        })
    
    return email_data

if __name__ == '__main__':
    # Авторизуемся и получаем доступ к Gmail API
    service = get_gmail_service()
    
    # Получаем непрочитанные сообщения
    unread_messages = get_unread_messages(service)
    print(unread_messages)


ModuleNotFoundError: No module named 'google.auth'